In [1]:
from extractGeometries import *

In [2]:
h_r = AreaHandler()
h_r.apply_file("data/osm/rec.osm.pbf")

In [3]:
h_r.get_buildings()

ValueError: invalid literal for int() with base 10: '010600000001000000010300000001000000050000003DBDAD4F946F41C0C4318111EA2020C040170D198F6F41C000C974E8F42020C075B1C45E836F41C0878494449B2020C03196E997886F41C021CA17B4902020C03DBDAD4F946F41C0C4318111EA2